In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
import math
from tqdm import tqdm
import toolmode.cfg_decoding as cfg_decoding
import toolmode.data.funcqa as funcqa
import toolmode.data.funcqa.runner as funcqa_runner

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = funcqa.load()
dataset

Saving the dataset (1/1 shards): 100%|██████████| 60/60 [00:00<00:00, 16674.94 examples/s]


Dataset({
    features: ['question', 'answer', 'func', 'operation', 'operation_num_params', 'prompt'],
    num_rows: 20
})

In [4]:
dataset[0]

{'question': 'A car travels 0.384 miles and then travels 1.63 more miles. How far did the car travel in total?',
 'answer': 2.014,
 'func': '<T>add(0.384, 1.63)=2.014',
 'operation': 'add',
 'operation_num_params': 2,
 'prompt': 'Answer the following question with the operator add:\n\nQ: A store sells 311 apples and 112 oranges. How many fruits are sold in total?\nA: The total number of fruits sold by the store is 311+112=<T>add(311, 112)=423.\n\nQ: A bakery made 201 loaves of bread in the morning and 31 in the afternoon. How many loaves of bread did they make in total?\nA: The total number of loaves of bread made is 201+31=<T>add(201, 31)=232.\n\nQ: A car traveled 118.01 miles in the morning and 451.96 miles in the afternoon. How many miles did the car travel in total?\nA: The car traveled 118.01+451.96=<T>add(118.01, 451.96)=569.97 miles in total.\n\nQ: A store sells 277 apples, 209 oranges, and 72 bananas. How many fruits are sold in total?\nA: The total number of fruits sold by the

In [5]:
print(dataset[0]["prompt"])

Answer the following question with the operator add:

Q: A store sells 311 apples and 112 oranges. How many fruits are sold in total?
A: The total number of fruits sold by the store is 311+112=<T>add(311, 112)=423.

Q: A bakery made 201 loaves of bread in the morning and 31 in the afternoon. How many loaves of bread did they make in total?
A: The total number of loaves of bread made is 201+31=<T>add(201, 31)=232.

Q: A car traveled 118.01 miles in the morning and 451.96 miles in the afternoon. How many miles did the car travel in total?
A: The car traveled 118.01+451.96=<T>add(118.01, 451.96)=569.97 miles in total.

Q: A store sells 277 apples, 209 oranges, and 72 bananas. How many fruits are sold in total?
A: The total number of fruits sold by the store is 277+209+72=<T>add(277, 209, 72)=558.

Q: A car travels 0.384 miles and then travels 1.63 more miles. How far did the car travel in total?
A:


In [6]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
cfg_decoder = cfg_decoding.CfgDecoder(MODEL_NAME, device_map="cuda:0")

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.37s/it]


In [7]:
results = {}

for i, row in tqdm(list(enumerate(dataset))):
    prompt = row["prompt"]
    answer_p1 = cfg_decoder.generate(prompt, max_new_tokens=500)
    answer_p2 = cfg_decoder.generate_constrained(prompt + answer_p1, max_new_tokens=100)
    
    operation = answer_p2.split("<T>")[1]
    result = funcqa_runner.evaluate_toolcall(operation)

    y_pred = float(result) if result is not None else float("nan")
    y_true = float(row["answer"])
    is_correct = math.isclose(y_pred, y_true, rel_tol=0.1)
    results[i] = {"y_true": y_true, "y_pred": y_pred, "is_correct": is_correct}
    
    print(row["question"])
    print(answer_p1)
    print(answer_p2)
    print(result)
    print(row["answer"])
    print(is_correct)
    print()

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will le

A car travels 0.384 miles and then travels 1.63 more miles. How far did the car travel in total?
The car traveled 0.384+1.63=
<T>add(0.384,1.63)
2.01
2.014
True



 10%|█         | 2/20 [00:19<02:52,  9.56s/it]

A marathon runner has completed 8.7832 miles so far. He has 5.3689 miles left to run. What is the total distance of the marathon?
The total distance of the marathon is 8.7832+5.3689=
<T>add(8.7832,5.3689)
14.15
14.1521
True



 15%|█▌        | 3/20 [00:28<02:33,  9.03s/it]

...

A store sells a total of 898.75 ounces of cereal. If they sell 43 boxes of cereal, how many ounces of cereal are in each box?
Each box of cereal contains 898.75/43=
<T>divide(898.75,43)
20.9
20.9012
True



 95%|█████████▌| 19/20 [02:51<00:08,  8.36s/it]

A swimming pool has a volume of 19,338.74 gallons. If the water can evenly fill 532 buckets, how many gallons of water are in each bucket?
Each bucket can hold 19,338.74/532=
<T>divide(19338.74,532)
36.35
36.351
True



100%|██████████| 20/20 [03:03<00:00,  9.19s/it]

A marathon is 26.2823 miles long. If a runner finishes the marathon in 216.3 minutes, what is the runner's average speed per mile?
The runner's average speed per mile is 26.2823/216.3=
<T>divide(26.2823,216.3000000000000333333333333333333333333333333333333333333333333333333333333333333333
None
8.2299
False



In [10]:
accuracy = sum([1 for r in results.values() if r["is_correct"]]) / len(results)
accuracy * 100

75.0